# Implementation of RBF neural network
In this section you should implement a RBF neural network that can be use as classifier. for do this you should fill missing part. just find optimal beta and write RBF training algorithm.

In [1]:
import random as rnd
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import math
def main():
    from dataloader import select_features
    import numpy as np
    import os
    from matplotlib.pyplot import imread
    val_num = 1000
    train_num = 49000
    test_num = 10000
    train_data, train_labels, test_data, test_labels,\
        class_names, n_train, n_test, n_class, n_features = select_features()
    X_train = train_data
    y_train = train_labels
    X_test = test_data
    y_test = test_labels
    # Subsample the data
    df_train = X_train
    df_test = X_test
    trueLabels = np.concatenate((y_train,y_test),axis=0)
    print ('Train data shape: ', X_train.shape)
    print ('Train labels shape: ', y_train.shape)
    print ('Test data shape: ', X_test.shape)
    print ('Test labels shape: ', y_test.shape)

    t=RBFNet()
    nlabels=10                #total number of labels
    nclusters=50              #number of clusters for k-means
    ksplits=10                #ksplits-fold cross validation

    #############################################################################
    def result(beta) :
        predict, center, centroidLabel = t.trainRBF(df_train, nclusters, beta, nlabels, trueLabels)
        predict = t.RBF(df_test, beta, center, centroidLabel, nlabels)
        res = (predict == y_test).mean() * 100
        print(res)
        return res
    
    results = np.array(list(map(lambda x: result(x), range(1,10))))
    print(results)
    optimalBeta = np.argmax(results)+1
    print('optimal beta :', optimalBeta)
    #############################################################################
    
    #Train
    print('training...')
    (predictedLabels_train,centers_train,centroidLabel_train)=t.trainRBF(df_train,nclusters,optimalBeta,nlabels,trueLabels)

    #Test
    print('testing...')
    predictedTestLabels=t.RBF(df_test,optimalBeta,centers_train,centroidLabel_train,nlabels)

    #Test accuracy
    testLabels=y_test
    accuracy=0
    for y in range(len(predictedTestLabels)):
        if predictedTestLabels[y] == testLabels[y]:
            accuracy+=1
    accuracy=(accuracy/len(predictedTestLabels))*100
    print('Percent accuracy on test data:',accuracy)


class RBFNet():
    def __init__(self):
        pass

    def calcKmean(self,data,n):
        kmeanz = KMeans(n_clusters=n).fit(data)
        centers=np.array(kmeanz.cluster_centers_)
        closest,_=pairwise_distances_argmin_min(kmeanz.cluster_centers_,data)
        closest=np.array(closest)
        return (centers,closest)

    def RBF(self,data,beta,centers,centroidLabels,nlabels):
#         print(X_train)
        #############################################################################
        score = lambda i : np.exp(-beta*1e-6*(np.sum((data - centers[i])**2, axis = 1)))
        scores = np.array(list(map(score, range(centroidLabels.shape[0])))).T
        choosenIndexes = np.array(list(map(lambda li: np.argmax(li), scores)))
        predictedLabels = centroidLabels[choosenIndexes]
        #############################################################################
        return predictedLabels

    def trainRBF(self,data,k,beta,nlabels,trueLabels):
        #k-means: Getting centroids and the row indices (in df_train) of the data points closest to the centroids
        t=RBFNet()
        (centers,indices)=t.calcKmean(data,k)
        #The label of each centroid according training data
        centroidLabel=np.zeros(len(centers))
        for x in range(len(centers)):
            centroidLabel[x]=trueLabels[indices[x]]
        predictedLabels=t.RBF(data,beta,centers,centroidLabel,nlabels)
        return (predictedLabels,centers,centroidLabel)


if __name__ == "__main__":
    main()


Train data shape:  (60000, 784)
Train labels shape:  (60000,)
Test data shape:  (10000, 784)
Test labels shape:  (10000,)
61.59
64.75
63.55
64.42999999999999
64.25999999999999
64.59
59.209999999999994
61.809999999999995
66.33
[61.59 64.75 63.55 64.43 64.26 64.59 59.21 61.81 66.33]
optimal beta : 9
training...
testing...
Percent accuracy on test data: 63.260000000000005
